In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
file_kondisi_fisik = "Dataset/Dataset Kondisi Fisik dan Kebutuhan Nutrisi.csv"
file_nutrisi_makanan = "Dataset/Dataset Informasi Nutrisi Makanan.csv"

kondisi_fisik = pd.read_csv(file_kondisi_fisik)
nutrisi_makanan = pd.read_csv(file_nutrisi_makanan)

In [3]:
kondisi_fisik.head(5)

,Usia,Jenis Kelamin,Berat (kg),Tinggi (cm),Aktivitas,Kalori Harian (kcal),Protein (g),Karbohidrat (g),Lemak (g)
0,1,Anak,10,75,Rendah,900,15,130,35
1,2,Anak,12,85,Rendah,1000,15,145,40
2,5,Anak,18,105,Sedang,1200,20,160,50
3,10,Anak,30,135,Sedang,1600,30,200,60
4,15,Pria,50,160,Tinggi,2500,60,300,70


In [4]:
nutrisi_makanan.head(5)

,Nama Makanan,Kalori (kcal),Protein (g),Karbohidrat (g),Lemak (g),Kategori
0,nasi jagung,230,5.0,50.0,3.0,Karbohidrat
1,nasi putih,130,2.7,28.2,0.3,Karbohidrat
2,nasi goreng,267,7.0,35.0,12.0,Karbohidrat
3,ayam goreng,260,28.4,0.1,15.5,Protein
4,tahu goreng,80,7.8,2.4,5.2,Protein


In [5]:
kondisi_fisik.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Usia                  24 non-null     int64 
 1   Jenis Kelamin         24 non-null     object
 2   Berat (kg)            24 non-null     int64 
 3   Tinggi (cm)           24 non-null     int64 
 4   Aktivitas             24 non-null     object
 5   Kalori Harian (kcal)  24 non-null     int64 
 6   Protein (g)           24 non-null     int64 
 7   Karbohidrat (g)       24 non-null     int64 
 8   Lemak (g)             24 non-null     int64 
dtypes: int64(7), object(2)
memory usage: 1.8+ KB


In [6]:
nutrisi_makanan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Nama Makanan     52 non-null     object 
 1   Kalori (kcal)    52 non-null     int64  
 2   Protein (g)      52 non-null     float64
 3   Karbohidrat (g)  52 non-null     float64
 4   Lemak (g)        52 non-null     float64
 5   Kategori         52 non-null     object 
dtypes: float64(3), int64(1), object(2)
memory usage: 2.6+ KB


In [7]:
# Gabungkan dataset menjadi fitur dan target dengan suffix untuk membedakan kolom yang sama
kondisi_fisik['Key'] = 1
nutrisi_makanan['Key'] = 1
data = pd.merge(kondisi_fisik, nutrisi_makanan, on='Key', suffixes=('_x', '_y')).drop('Key', axis=1)
data.head(5)

,Usia,Jenis Kelamin,Berat (kg),Tinggi (cm),Aktivitas,Kalori Harian (kcal),Protein (g)_x,Karbohidrat (g)_x,Lemak (g)_x,Nama Makanan,Kalori (kcal),Protein (g)_y,Karbohidrat (g)_y,Lemak (g)_y,Kategori
0,1,Anak,10,75,Rendah,900,15,130,35,nasi jagung,230,5.0,50.0,3.0,Karbohidrat
1,1,Anak,10,75,Rendah,900,15,130,35,nasi putih,130,2.7,28.2,0.3,Karbohidrat
2,1,Anak,10,75,Rendah,900,15,130,35,nasi goreng,267,7.0,35.0,12.0,Karbohidrat
3,1,Anak,10,75,Rendah,900,15,130,35,ayam goreng,260,28.4,0.1,15.5,Protein
4,1,Anak,10,75,Rendah,900,15,130,35,tahu goreng,80,7.8,2.4,5.2,Protein


In [8]:
def hitung_bmi(berat, tinggi):
    tinggi_m = tinggi / 100
    return berat / (tinggi_m ** 2)

def kategori_bmi(bmi):
    if bmi < 18.5:
        return "Kurus"
    elif 18.5 <= bmi < 25:
        return "Normal"
    elif 25 <= bmi < 30:
        return "Kelebihan Berat Badan"
    else:
        return "Obesitas"

def tentukan_kategori_bmi(berat, tinggi):
    bmi = hitung_bmi(berat, tinggi)
    kategori = kategori_bmi(bmi)
    return bmi, kategori

In [ ]:
# Fitur yang digunakan adalah: Usia, Berat (kg), Tinggi (cm), dan Lemak (g)
features = ['Usia', 'Berat (kg)', 'Tinggi (cm)', 'Lemak (g)_x']
# Target (nutrisi yang akan direkomendasikan)
target = ['Kalori (kcal)_y', 'Protein (g)_y', 'Karbohidrat (g)_y', 'Lemak (g)_y']


In [10]:
features = ['Usia', 'Berat (kg)', 'Tinggi (cm)', 'Lemak (g)_x']
target = ['Kalori (kcal)_y', 'Protein (g)_y', 'Karbohidrat (g)_y', 'Lemak (g)_y']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features)
    ]
)

In [11]:
# KNN Model untuk rekomendasi makanan
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', NearestNeighbors(n_neighbors=5))])
pipeline.fit(data[features])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Usia', 'Berat (kg)',
                                                   'Tinggi (cm)',
                                                   'Lemak (g)_x'])])),
                ('model', NearestNeighbors())])

In [13]:
def get_recommendations(usia, berat, tinggi, lemak):
    input_user = pd.DataFrame({
        'Usia': [usia],
        'Berat (kg)': [berat],
        'Tinggi (cm)': [tinggi],
        'Lemak (g)_x': [lemak]
    })
    
    bmi, kategori_bmi = tentukan_kategori_bmi(berat, tinggi)
    
    input_transformed = pipeline['preprocessor'].transform(input_user)
    distances, indices = pipeline['model'].kneighbors(input_transformed)
    
    rekomendasi = data.iloc[indices[0]]
    
    print(f"BMI: {bmi:.2f}")
    print(f"Kategori BMI: {kategori_bmi}")
    print("\nRekomendasi Makanan:")
    print(rekomendasi[['Nama Makanan', 'Kalori (kcal)', 'Protein (g)_y', 'Karbohidrat (g)_y', 'Lemak (g)_y']])

# Contoh penggunaan
get_recommendations(30, 50, 160, 50)

BMI: 19.53
Kategori BMI: Normal

Rekomendasi Makanan:
     Nama Makanan  Kalori (kcal)  Protein (g)_y  Karbohidrat (g)_y  \
577  tempe goreng            158           10.0                8.0   
580         pecel            120            4.0               12.0   
578   sayur lodeh            100            3.5               10.0   
576   tahu goreng             80            7.8                2.4   
579     gado-gado            150            5.0               15.0   

     Lemak (g)_y  
577         10.0  
580          6.0  
578          5.0  
576          5.2  
579          8.0  
